In [8]:
import numpy as np
import jax
import jax.numpy as jnp
import haiku as hk

In [28]:
key = jax.random.PRNGKey(42)
mask = jax.random.bernoulli(key, 0.5, shape=[10, 10])

In [29]:
jnp.ones([10, 10], dtype=jnp.float32) * mask

DeviceArray([[0., 0., 0., 1., 1., 1., 1., 0., 1., 1.],
             [0., 1., 0., 0., 0., 0., 0., 1., 1., 1.],
             [1., 1., 0., 0., 0., 1., 0., 1., 1., 0.],
             [1., 1., 1., 1., 1., 0., 0., 1., 1., 1.],
             [0., 0., 0., 1., 1., 0., 0., 0., 0., 0.],
             [1., 0., 0., 0., 1., 1., 1., 0., 0., 1.],
             [1., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
             [1., 0., 1., 0., 1., 1., 1., 0., 0., 1.],
             [0., 0., 1., 0., 0., 0., 0., 1., 1., 1.],
             [0., 1., 1., 1., 1., 1., 1., 1., 0., 1.]], dtype=float32)